In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
from statsmodels.formula.api import ols
import statsmodels.api as sm 
import warnings
warnings.filterwarnings('ignore')
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from numpy import mean, std
from sklearn.manifold import MDS
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

# Load into Python
kw8322228 = pd.read_csv("clicksdata.kw8322228.csv", index_col = 0)
kw8322392 = pd.read_csv("clicksdata.kw8322392.csv", index_col = 0)
kw8322393 = pd.read_csv("clicksdata.kw8322393.csv", index_col = 0)
kw8322445 = pd.read_csv("clicksdata.kw8322445.csv", index_col = 0)
kwltvconv = pd.read_excel("kw-ltv-conv.xlsx")

Part A: Estimate the alpha and beta parameters for each of these four keywords for this firm

In [111]:
# Defining all functions
def clicks(x, alpha, beta):
  return alpha * (1 - np.exp(-beta * x))

def profit(bid, alpha, beta, ltv, conv):
    return -clicks(bid, alpha, beta) * (ltv * conv - bid)

def expenditure(bid, alpha, beta, ltv, conv):
    return clicks(bid, alpha, beta) * bid


In [112]:
def estimate_parameters(data):
    x = data.iloc[:, 0]
    y = data.iloc[:, 1]
    alpha_initial = y.iloc[-1]
    beta_initial  = 1 / np.mean(x)
    params, covariance = curve_fit(clicks, x, y, p0 = [alpha_initial, beta_initial])
    rss = np.sum((y - clicks(x, *params)) ** 2)
    return params, rss

kwltvconv['α'] = np.nan
kwltvconv['β'] = np.nan
kwltvconv['RSS'] = np.nan
i = 0
for data in [kw8322228, kw8322392, kw8322393, kw8322445]:
  params, rss = estimate_parameters(data)
  kwltvconv.loc[i, 'α'] = params[0]
  kwltvconv.loc[i, 'β'] = params[1]
  kwltvconv.loc[i, 'RSS'] = rss
  i += 1

In [113]:
kwltvconv

,keyword,ltv,conv.rate,α,β,RSS
0,kw8322228,354,0.30,74.090862,0.039449,229.814751
1,kw8322392,181,0.32,156.439803,0.150083,383.573004
2,kw8322393,283,0.30,104.799293,0.079717,216.308754
3,kw8322445,107,0.30,188.111279,0.432292,565.690580


Part B: Assume that the LTV dollar value and the conversion rate values for each of the keywords for this firm are as shown in the table below.

In [125]:
kwltvconv['nobudget_optbid'] = np.nan
kwltvconv['nobudget_optprofit'] = np.nan
kwltvconv['nobudget_optexp'] = np.nan

for i in range(4):
    alpha = kwltvconv.iloc[i, 3]
    beta = kwltvconv.iloc[i, 4]
    ltv = kwltvconv.iloc[i, 1]
    conv = kwltvconv.iloc[i, 2]

    opt_bid = minimize(profit, 
                    x0 = 1, 
                    args = (alpha, beta, ltv, conv), 
                    bounds = Bounds(lb = 0, ub = np.inf)).x[0]
    
    kwltvconv.loc[i, 'nobudget_optbid'] = opt_bid
    kwltvconv.loc[i, 'nobudget_optprofit'] = -profit(opt_bid, alpha, beta, ltv, conv)
    kwltvconv.loc[i, 'nobudget_optexp'] = expenditure(opt_bid, alpha, beta, ltv, conv)

In [126]:
kwltvconv

,keyword,ltv,conv.rate,α,β,RSS,nobudget_optbid,nobudget_optprofit,nobudget_optexp
0,kw8322228,354,0.30,74.090862,0.039449,229.814751,34.127617,3950.456962,1870.615060
1,kw8322392,181,0.32,156.439803,0.150083,383.573004,13.563446,6032.902219,1844.754341
2,kw8322393,283,0.30,104.799293,0.079717,216.308754,22.433866,5451.614108,1957.873313
3,kw8322445,107,0.30,188.111279,0.432292,565.690580,5.816956,4544.188925,1005.718637


In [127]:
# # Import Gurobi
# from gurobipy import *

# # Input Variables
# ltv = kwltvconv.iloc[0, 1].astype(float)
# conv = kwltvconv.iloc[0, 2].astype(float)
# alpha = kwltvconv.iloc[0, 3].astype(float)
# beta = kwltvconv.iloc[0, 4].astype(float)

# # Initialise
# m = Model("Media Mix Model")
# m.setParam('OutputFlag', 0)
# m.setParam('NonConvex', 2)

# # Define Variables
# bid = m.addVar(vtype = GRB.CONTINUOUS, name = "bid", lb = 0)

# # Define Objective Function
# m.setObjective(alpha * (1 - np.exp(-beta * bid)) * (ltv * conv - bid), GRB.MAXIMIZE)

# # Optimize
# m.update()
# m.optimize()

# # Output
# print('Optimal Bid:', bid.x)

Part C: Assume now that you have a budget constraint of $3000 across these four keywords.

In [145]:
def total_expenditure(params):
    bid1, bid2, bid3, bid4 = params
    return expenditure(bid1, alpha = kwltvconv.iloc[0, 3], beta = kwltvconv.iloc[0, 4], ltv = kwltvconv.iloc[0, 1], conv = kwltvconv.iloc[0, 2]) + \
           expenditure(bid2, alpha = kwltvconv.iloc[1, 3], beta = kwltvconv.iloc[1, 4], ltv = kwltvconv.iloc[1, 1], conv = kwltvconv.iloc[1, 2]) + \
           expenditure(bid3, alpha = kwltvconv.iloc[2, 3], beta = kwltvconv.iloc[2, 4], ltv = kwltvconv.iloc[2, 1], conv = kwltvconv.iloc[2, 2]) + \
           expenditure(bid4, alpha = kwltvconv.iloc[3, 3], beta = kwltvconv.iloc[3, 4], ltv = kwltvconv.iloc[3, 1], conv = kwltvconv.iloc[3, 2])

def total_profit(params):
    bid1, bid2, bid3, bid4 = params
    return profit(bid1, alpha = kwltvconv.iloc[0, 3], beta = kwltvconv.iloc[0, 4], ltv = kwltvconv.iloc[0, 1], conv = kwltvconv.iloc[0, 2]) + \
           profit(bid2, alpha = kwltvconv.iloc[1, 3], beta = kwltvconv.iloc[1, 4], ltv = kwltvconv.iloc[1, 1], conv = kwltvconv.iloc[1, 2]) + \
           profit(bid3, alpha = kwltvconv.iloc[2, 3], beta = kwltvconv.iloc[2, 4], ltv = kwltvconv.iloc[2, 1], conv = kwltvconv.iloc[2, 2]) + \
           profit(bid4, alpha = kwltvconv.iloc[3, 3], beta = kwltvconv.iloc[3, 4], ltv = kwltvconv.iloc[3, 1], conv = kwltvconv.iloc[3, 2])

# Input Variables
budget = 3000
budget_constraint_object = NonlinearConstraint(total_expenditure, 0, budget)


opt_bid = minimize(total_profit, 
                    x0 = [1, 1, 1, 1],
                    bounds = Bounds(lb = 0, ub = np.inf),
                    method='trust-constr',
                    constraints = budget_constraint_object)

kwltvconv['budget_optbid'] = opt_bid.x
kwltvconv['budget_optprofit'] = -profit(kwltvconv.budget_optbid, kwltvconv.α, kwltvconv.β, kwltvconv.ltv, kwltvconv['conv.rate'])
kwltvconv['budget_optexp'] = expenditure(kwltvconv.budget_optbid, kwltvconv.α, kwltvconv.β, kwltvconv.ltv, kwltvconv['conv.rate'])

In [149]:
#percentage reduction in expenditure
kwltvconv['exp_reduction'] = (kwltvconv['nobudget_optexp'] - kwltvconv['budget_optexp']) / kwltvconv['nobudget_optexp'] * 100

In [150]:
kwltvconv

,keyword,ltv,conv.rate,α,β,RSS,nobudget_optbid,nobudget_optprofit,nobudget_optexp,budget_optbid,budget_optprofit,budget_optexp,exp_reduction
0,kw8322228,354,0.30,74.090862,0.039449,229.814751,34.127617,3950.456962,1870.615060,17.924261,3315.507393,673.208961,64.011358
1,kw8322392,181,0.32,156.439803,0.150083,383.573004,13.563446,6032.902219,1844.754341,8.118451,5487.232096,894.506849,51.510788
2,kw8322393,283,0.30,104.799293,0.079717,216.308754,22.433866,5451.614108,1957.873313,12.828288,4836.614287,860.885350,56.029568
3,kw8322445,107,0.30,188.111279,0.432292,565.690580,5.816956,4544.188925,1005.718637,3.775700,4286.482744,571.398837,43.185021
